In [2]:
import pandas as pd
from datetime import datetime
import requests
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dateutil.rrule import rrule, MONTHLY

In [ ]:
def get_first_and_last_days(start_year, end_year):
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)

    first_days = list(rrule(freq=MONTHLY, dtstart=start_date, until=end_date))
    last_days = [day + relativedelta(months=1, days=-1) for day in first_days]

    return first_days, last_days

## CREATE THE LIST OF DATES - FIRST AND LAST DAY OF THE MONTHS STARTING 2013-01-01
The api responds with a max of 280 hourly records so I am batching the calls by month. 

In [5]:
# Set the start date to January 1, 2013
start_date = datetime(2020, 1, 1)

# Get today's date for the upper limit of the date range
end_date = datetime(2024, 3, 31)

# Generate the first day of each month from January 2013 to current month
first_days = list(rrule(freq=MONTHLY, dtstart=start_date, until=end_date))

# Calculate the last day of each month using the first day of each month
last_days = [day + relativedelta(months=1, days=-1) for day in first_days]

# Printing the first and last days
for first, last in zip(first_days, last_days):
    print(f"First day: {first.date()}, Last day: {last.date()}")

First day: 2020-01-01, Last day: 2020-01-31
First day: 2020-02-01, Last day: 2020-02-29
First day: 2020-03-01, Last day: 2020-03-31
First day: 2020-04-01, Last day: 2020-04-30
First day: 2020-05-01, Last day: 2020-05-31
First day: 2020-06-01, Last day: 2020-06-30
First day: 2020-07-01, Last day: 2020-07-31
First day: 2020-08-01, Last day: 2020-08-31
First day: 2020-09-01, Last day: 2020-09-30
First day: 2020-10-01, Last day: 2020-10-31
First day: 2020-11-01, Last day: 2020-11-30
First day: 2020-12-01, Last day: 2020-12-31
First day: 2021-01-01, Last day: 2021-01-31
First day: 2021-02-01, Last day: 2021-02-28
First day: 2021-03-01, Last day: 2021-03-31
First day: 2021-04-01, Last day: 2021-04-30
First day: 2021-05-01, Last day: 2021-05-31
First day: 2021-06-01, Last day: 2021-06-30
First day: 2021-07-01, Last day: 2021-07-31
First day: 2021-08-01, Last day: 2021-08-31
First day: 2021-09-01, Last day: 2021-09-30
First day: 2021-10-01, Last day: 2021-10-31
First day: 2021-11-01, Last day:

In [6]:
api_endpoint = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=bf2b1f005ae542e89b1123837241204&q=New York City&format=json&extra=isDayTime&date=2013-01-01&enddate=2024-03-31&tp=tp=1'

base_url = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx'
key = 'bf2b1f005ae542e89b1123837241204'

df_weather = pd.DataFrame()

for start_date, end_date in zip(first_days, last_days):
    url = f'{base_url}?key={key}&q=New York City&format=json&extra=isDayTime&date={start_date}&enddate={end_date}&tp=tp=1'
    response = requests.get(url)

    if not response.ok:
        raise Exception(f"Failed to fetch data from API: {response.status_code}")

    rows = []

    for entry in response.json()['data']['weather']:
        date = entry['date']
        for hour in entry['hourly']:
            row = {
                'date': date,
                'time': hour['time'],
                'tempC': hour['tempC'],
                'tempF': hour['tempF'],
                'windspeedMiles': hour['windspeedMiles'],
                'windspeedKmph': hour['windspeedKmph'],
                'winddirDegree': hour['winddirDegree'],
                'winddir16Point': hour['winddir16Point'],
                'weatherCode': hour['weatherCode'],
                'weatherDesc': hour['weatherDesc'][0]['value'],
                'precipMM': hour['precipMM'],
                'humidity': hour['humidity'],
                'visibility': hour['visibility'],
                'pressure': hour['pressure'],
                'cloudcover': hour['cloudcover'],
            }
            rows.append(row)

    df_batch = pd.DataFrame(rows)
    print(df_batch.shape)
    df_weather = pd.concat([df_weather, df_batch], ignore_index=True)
    print(df_weather.shape)


print(df_weather.head())


(248, 15)
(248, 15)
(232, 15)
(480, 15)
(248, 15)
(728, 15)
(240, 15)
(968, 15)
(248, 15)
(1216, 15)
(240, 15)
(1456, 15)
(248, 15)
(1704, 15)
(248, 15)
(1952, 15)
(240, 15)
(2192, 15)
(248, 15)
(2440, 15)
(240, 15)
(2680, 15)
(248, 15)
(2928, 15)
(248, 15)
(3176, 15)
(224, 15)
(3400, 15)
(248, 15)
(3648, 15)
(240, 15)
(3888, 15)
(248, 15)
(4136, 15)
(240, 15)
(4376, 15)
(248, 15)
(4624, 15)
(248, 15)
(4872, 15)
(240, 15)
(5112, 15)
(248, 15)
(5360, 15)
(240, 15)
(5600, 15)
(248, 15)
(5848, 15)
(248, 15)
(6096, 15)
(224, 15)
(6320, 15)
(248, 15)
(6568, 15)
(240, 15)
(6808, 15)
(248, 15)
(7056, 15)
(240, 15)
(7296, 15)
(248, 15)
(7544, 15)
(248, 15)
(7792, 15)
(240, 15)
(8032, 15)
(248, 15)
(8280, 15)
(240, 15)
(8520, 15)
(248, 15)
(8768, 15)
(248, 15)
(9016, 15)
(224, 15)
(9240, 15)
(248, 15)
(9488, 15)
(240, 15)
(9728, 15)
(248, 15)
(9976, 15)
(240, 15)
(10216, 15)
(248, 15)
(10464, 15)
(248, 15)
(10712, 15)
(240, 15)
(10952, 15)
(248, 15)
(11200, 15)
(240, 15)
(11440, 15)
(248, 15)
(

In [7]:
df_weather.shape

(12416, 15)

In [21]:
df_weather.to_parquet('/home/lulu/projects/data_talks/nyc-collisions/mage/weather_data.parquet', index=False)

In [16]:
df_weather2 = df_weather.copy()

In [17]:
df_weather2['time'] = df_weather2['time'].apply(lambda x: x.zfill(4))


In [18]:
df_weather2['time']  = pd.to_datetime(df_weather2['time'], format='%H%M').dt.time

In [19]:
df_weather2.head(20)

,date,time,tempC,tempF,windspeedMiles,windspeedKmph,winddirDegree,winddir16Point,weatherCode,weatherDesc,precipMM,humidity,visibility,pressure,cloudcover
0,2013-01-01,00:00,1,34,12,20,252,WSW,116,Partly cloudy,0.0,86,9,1013,57
1,2013-01-01,03:00,1,34,14,22,266,W,119,Cloudy,0.0,88,10,1012,76
2,2013-01-01,06:00,0,33,10,16,250,WSW,113,Sunny,0.0,90,10,1013,16
3,2013-01-01,09:00,2,36,12,20,275,W,116,Partly cloudy,0.0,88,10,1013,49
4,2013-01-01,12:00,3,38,9,15,289,WNW,116,Partly cloudy,0.0,77,10,1012,29
5,2013-01-01,15:00,3,37,12,19,308,NW,116,Partly cloudy,0.0,78,10,1012,54
6,2013-01-01,18:00,-0,31,11,17,326,NNW,113,Clear,0.0,74,10,1015,11
7,2013-01-01,21:00,-3,26,11,18,318,NW,113,Clear,0.0,78,10,1016,17
8,2013-01-02,00:00,-6,21,12,19,321,NW,113,Clear,0.0,78,10,1017,12
9,2013-01-02,03:00,-7,19,11,18,327,NNW,113,Clear,0.0,79,10,1017,11


### RESPONSE STRUCTURE 
I need to manually flatten as it isnt working with normalize and dlt doesnt like the format of the response either but I dont know why

In [ ]:
{
    "data": {
        "request": [
            {
                "type": "City",
                "query": "New York, United States of America"
            }
        ],
        "weather": [
            {
                "date": "2013-01-01",
                "astronomy": [
                    {
                        "sunrise": "07:20 AM",
                        "sunset": "04:39 PM",
                        "moonrise": "09:10 PM",
                        "moonset": "09:30 AM",
                        "moon_phase": "Waning Gibbous",
                        "moon_illumination": "68"
                    }
                ],
                "maxtempC": "3",
                "maxtempF": "38",
                "mintempC": "-5",
                "mintempF": "23",
                "avgtempC": "1",
                "avgtempF": "33",
                "totalSnow_cm": "0.0",
                "sunHour": "9.0",
                "uvIndex": "2",
                "hourly": [
                    {
                        "time": "0",
                        "tempC": "1",
                        "tempF": "34",
                        "windspeedMiles": "12",
                        "windspeedKmph": "20",
                        "winddirDegree": "252",
                        "winddir16Point": "WSW",
                        "weatherCode": "116",
                        "weatherIconUrl": [
                            {
                                "value": "https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0004_black_low_cloud.png"
                            }
                        ],
                        "weatherDesc": [
                            {
                                "value": "Partly cloudy"
                            }
                        ],
                        "precipMM": "0.0",
                        "precipInches": "0.0",
                        "humidity": "86",
                        "visibility": "9",
                        "visibilityMiles": "5",
                        "pressure": "1013",
                        "pressureInches": "30",
                        "cloudcover": "57",
                        "HeatIndexC": "1",
                        "HeatIndexF": "34",
                        "DewPointC": "-1",
                        "DewPointF": "30",
                        "WindChillC": "-4",
                        "WindChillF": "25",
                        "WindGustMiles": "21",
                        "WindGustKmph": "34",
                        "FeelsLikeC": "-4",
                        "FeelsLikeF": "25",
                        "uvIndex": "1"
                    },
                    {
                        "time": "300",
                        "tempC": "1",
                        "tempF": "34",
                        "windspeedMiles": "14",
                        "windspeedKmph": "22",
                        "winddirDegree": "266",
                        "winddir16Point": "W",
                        "weatherCode": "119",
                        "weatherIconUrl": [
                            {
                                "value": "https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0003_white_cloud.png"
                            }
                        ],
                        "weatherDesc": [
                            {
                                "value": "Cloudy"
                            }
                        ],
                        "precipMM": "0.0",
                        "precipInches": "0.0",
                        "humidity": "88",
                        "visibility": "10",
                        "visibilityMiles": "6",
                        "pressure": "1012",
                        "pressureInches": "30",
                        "cloudcover": "76",
                        "HeatIndexC": "1",
                        "HeatIndexF": "34",
                        "DewPointC": "-1",
                        "DewPointF": "31",
                        "WindChillC": "-4",
                        "WindChillF": "24",
                        "WindGustMiles": "23",
                        "WindGustKmph": "37",
                        "FeelsLikeC": "-4",
                        "FeelsLikeF": "24",
                        "uvIndex": "1"
                    },
                    {
                        "time": "600",
                        "tempC": "0",
                        "tempF": "33",
                        "windspeedMiles": "10",
                        "windspeedKmph": "16",
                        "winddirDegree": "250",
                        "winddir16Point": "WSW",
                        "weatherCode": "113",
                        "weatherIconUrl": [
                            {
                                "value": "https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0001_sunny.png"
                            }
                        ],
                        "weatherDesc": [
                            {
                                "value": "Sunny"
                            }
                        ],
                        "precipMM": "0.0",
                        "precipInches": "0.0",
                        "humidity": "90",
                        "visibility": "10",
                        "visibilityMiles": "6",
                        "pressure": "1013",
                        "pressureInches": "30",
                        "cloudcover": "16",
                        "HeatIndexC": "0",
                        "HeatIndexF": "33",
                        "DewPointC": "-1",
                        "DewPointF": "30",
                        "WindChillC": "-4",
                        "WindChillF": "24",
                        "WindGustMiles": "17",
                        "WindGustKmph": "28",
                        "FeelsLikeC": "-4",
                        "FeelsLikeF": "24",
                        "uvIndex": "2"
                    },
                    {
                        "time": "900",
                        "tempC": "2",
                        "tempF": "36",
                        "windspeedMiles": "12",
                        "windspeedKmph": "20",
                        "winddirDegree": "275",
                        "winddir16Point": "W",
                        "weatherCode": "116",
                        "weatherIconUrl": [
                            {
                                "value": "https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0002_sunny_intervals.png"
                            }
                        ],
                        "weatherDesc": [
                            {
                                "value": "Partly cloudy"
                            }
                        ],
                        "precipMM": "0.0",
                        "precipInches": "0.0",
                        "humidity": "88",
                        "visibility": "10",
                        "visibilityMiles": "6",
                        "pressure": "1013",
                        "pressureInches": "30",
                        "cloudcover": "49",
                        "HeatIndexC": "2",
                        "HeatIndexF": "36",
                        "DewPointC": "1",
                        "DewPointF": "33",
                        "WindChillC": "-2",
                        "WindChillF": "28",
                        "WindGustMiles": "18",
                        "WindGustKmph": "29",
                        "FeelsLikeC": "-2",
                        "FeelsLikeF": "28",
                        "uvIndex": "2"
                    },
                    {
                        "time": "1200",
                        "tempC": "3",
                        "tempF": "38",
                        "windspeedMiles": "9",
                        "windspeedKmph": "15",
                        "winddirDegree": "289",
                        "winddir16Point": "WNW",
                        "weatherCode": "116",
                        "weatherIconUrl": [
                            {
                                "value": "https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0002_sunny_intervals.png"
                            }
                        ],
                        "weatherDesc": [
                            {
                                "value": "Partly cloudy"
                            }
                        ],
                        "precipMM": "0.0",
                        "precipInches": "0.0",
                        "humidity": "77",
                        "visibility": "10",
                        "visibilityMiles": "6",
                        "pressure": "1012",
                        "pressureInches": "30",
                        "cloudcover": "29",
                        "HeatIndexC": "3",
                        "HeatIndexF": "38",
                        "DewPointC": "-0",
                        "DewPointF": "32",
                        "WindChillC": "-0",
                        "WindChillF": "31",
                        "WindGustMiles": "17",
                        "WindGustKmph": "27",
                        "FeelsLikeC": "-0",
                        "FeelsLikeF": "31",
                        "uvIndex": "2"
                    },
                    {
                        "time": "1500",
                        "tempC": "3",
                        "tempF": "37",
                        "windspeedMiles": "12",
                        "windspeedKmph": "19",
                        "winddirDegree": "308",
                        "winddir16Point": "NW",
                        "weatherCode": "116",
                        "weatherIconUrl": [
                            {
                                "value": "https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0002_sunny_intervals.png"
                            }
                        ],
                        "weatherDesc": [
                            {
                                "value": "Partly cloudy"
                            }
                        ],
                        "precipMM": "0.0",
                        "precipInches": "0.0",
                        "humidity": "78",
                        "visibility": "10",
                        "visibilityMiles": "6",
                        "pressure": "1012",
                        "pressureInches": "30",
                        "cloudcover": "54",
                        "HeatIndexC": "3",
                        "HeatIndexF": "37",
                        "DewPointC": "-1",
                        "DewPointF": "30",
                        "WindChillC": "-2",
                        "WindChillF": "28",
                        "WindGustMiles": "19",
                        "WindGustKmph": "30",
                        "FeelsLikeC": "-2",
                        "FeelsLikeF": "28",
                        "uvIndex": "2"
                    },
                    {
                        "time": "1800",
                        "tempC": "-0",
                        "tempF": "31",
                        "windspeedMiles": "11",
                        "windspeedKmph": "17",
                        "winddirDegree": "326",
                        "winddir16Point": "NNW",
                        "weatherCode": "113",
                        "weatherIconUrl": [
                            {
                                "value": "https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0008_clear_sky_night.png"
                            }
                        ],
                        "weatherDesc": [
                            {
                                "value": "Clear"
                            }
                        ],
                        "precipMM": "0.0",
                        "precipInches": "0.0",
                        "humidity": "74",
                        "visibility": "10",
                        "visibilityMiles": "6",
                        "pressure": "1015",
                        "pressureInches": "30",
                        "cloudcover": "11",
                        "HeatIndexC": "-0",
                        "HeatIndexF": "31",
                        "DewPointC": "-6",
                        "DewPointF": "22",
                        "WindChillC": "-5",
                        "WindChillF": "22",
                        "WindGustMiles": "20",
                        "WindGustKmph": "33",
                        "FeelsLikeC": "-5",
                        "FeelsLikeF": "22",
                        "uvIndex": "1"
                    },
                    {
                        "time": "2100",
                        "tempC": "-3",
                        "tempF": "26",
                        "windspeedMiles": "11",
                        "windspeedKmph": "18",
                        "winddirDegree": "318",
                        "winddir16Point": "NW",
                        "weatherCode": "113",
                        "weatherIconUrl": [
                            {
                                "value": "https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0008_clear_sky_night.png"
                            }
                        ],
                        "weatherDesc": [
                            {
                                "value": "Clear"
                            }
                        ],
                        "precipMM": "0.0",
                        "precipInches": "0.0",
                        "humidity": "78",
                        "visibility": "10",
                        "visibilityMiles": "6",
                        "pressure": "1016",
                        "pressureInches": "30",
                        "cloudcover": "17",
                        "HeatIndexC": "-3",
                        "HeatIndexF": "26",
                        "DewPointC": "-8",
                        "DewPointF": "18",
                        "WindChillC": "-9",
                        "WindChillF": "16",
                        "WindGustMiles": "21",
                        "WindGustKmph": "34",
                        "FeelsLikeC": "-9",
                        "FeelsLikeF": "16",
                        "uvIndex": "1"
                    }
                ]
            },

## hourly data

In [ ]:
                    {
                        "time": "0",
                        "tempC": "1",
                        "tempF": "34",
                        "windspeedMiles": "12",
                        "windspeedKmph": "20",
                        "winddirDegree": "252",
                        "winddir16Point": "WSW",
                        "weatherCode": "116",
                        "weatherIconUrl": [
                            {
                                "value": "https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0004_black_low_cloud.png"
                            }
                        ],
                        "weatherDesc": [
                            {
                                "value": "Partly cloudy"
                            }
                        ],
                        "precipMM": "0.0",
                        "precipInches": "0.0",
                        "humidity": "86",
                        "visibility": "9",
                        "visibilityMiles": "5",
                        "pressure": "1013",
                        "pressureInches": "30",
                        "cloudcover": "57",
                        "HeatIndexC": "1",
                        "HeatIndexF": "34",
                        "DewPointC": "-1",
                        "DewPointF": "30",
                        "WindChillC": "-4",
                        "WindChillF": "25",
                        "WindGustMiles": "21",
                        "WindGustKmph": "34",
                        "FeelsLikeC": "-4",
                        "FeelsLikeF": "25",
                        "uvIndex": "1"
                    },